In [22]:
import pandas as pd
import numpy as np
import parameters as P
import sensitivity_parameters as SP

import helper_functions as hf

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
io_data = pd.read_csv(P.io_table_dollars_path).fillna(0)

In [27]:
water_consumption = hf.csv_dict_list(P.water_consumption_path)

In [28]:
water_consumption

{'1-bromoalkane.kg': 0.0,
 '1-methylimidazole.kg': 0.0,
 '1_chlorobutane.kg': 0.0,
 '1_chloroethane.kg': 0.0,
 'acetic.acid.kg': 0.0,
 'acetone.kg': 0.0,
 'alpha_amylase.kg': 0.0,
 'ammo.chloride.kg': 0.0,
 'ammonia.kg': 11.0,
 'ammonia_brazil.kg': 0.0,
 'ammonium.sulfate.kg': 0.0,
 'atrazine.kg': 7.3,
 'atrazine_brazil.kg': 0.0,
 'barge.mt_km': 0.0,
 'bromine.kg': 0.0,
 'butadiene.kg': 0.0,
 'butadiene_via_butane.kg': 0.0,
 'butane.MJ': 0.0,
 'caco3.kg': 0.0,
 'caco3_brazil.kg': 0.0,
 'cellulase.kg': 0.0,
 'cement.kg': 0.0,
 'char.MJ': 0.0,
 'cholinium.hydroxide.kg': 0.56,
 'cholinium.lysinate.kg': 0.0,
 'citric.acid.kg': 0.0,
 'coal.MJ': 0.0573,
 'concrete.cu.m': 0.0,
 'corn.bushel': 0.0,
 'corn.stover.kg': 0.0,
 'corn.stover.transportation.kg': 0.0,
 'corn_grain_ethanol.MJ': 0.0,
 'corn_grain_ethanol_rolledup.MJ': 0.0,
 'corn_oil.kg': 0.0,
 'corn_starch.kg': 0.0,
 'corn_stover_ethanol_rolledup.MJ': 0.0,
 'crudeoil.MJ': 0.0755,
 'cs.ethanol.chly.distribution.kg': 0.0,
 'cs.ethanol.ch

In [17]:
cost = hf.csv_dict_list(P.cost_impact_path)
etoh_feed_stream_mass_kg = 1 

In [18]:
y = {}
for item in cost.keys():
    y.update({item:0})
    
y_cred = {}
for item in cost.keys():
    y_cred.update({item:0})

In [19]:
m = {} 
for scenario in P.scenario_range:
    new_data = np.zeros([8,3])
    m[scenario] = pd.DataFrame(new_data, columns=P.selectivity, index=P.processes)
    
n = {} 
for scenario in P.scenario_range:
    new_data = np.zeros([1,3])
    n[scenario] = pd.DataFrame(new_data, columns=P.selectivity, index=['electricity_credit'])

In [20]:
for selectivity in P.selectivity:
    for scenario in P.scenario_range:
        y["lysine.us.kg"] = cost["lysine.us.kg"] * (SP.other_params[selectivity]['chlys_amount'][scenario] * 
                            SP.common_params['chlys_percent'][scenario]) 
        # 58% lysine + 42% ChOH for Chylys production (Stoichiometry) 
        y["cholinium.hydroxide.kg"] = (cost["cholinium.hydroxide.kg"] * 
                                       SP.other_params[selectivity]['chlys_amount'][scenario] * 
                                       SP.common_params['cholinium_percent'][scenario])  
        # 58% lysine + 42% ChOH for Chylys production (Stoichiometry) 
        y["cellulase.kg"] = cost["cellulase.kg"] * SP.common_params['enzyme'][scenario]
        y["csl.kg"] = cost["csl.kg"] * SP.other_params[selectivity]['csl.kg'][scenario]
        y["farmedstover.kg"] = cost["farmedstover.kg"] * SP.other_params[selectivity]['feedstock'][scenario]  
        y["dap.kg"] = cost["dap.kg"] * SP.other_params[selectivity]['dap.kg'][scenario] 
        y["h2so4.kg"] = cost["h2so4.kg"] * SP.other_params[selectivity]['h2so4.kg'][scenario]
        y["naturalgas.MJ"] = cost["naturalgas.MJ"] * (hf.FuelConvertMJ(
                            SP.other_params[selectivity]['ng_input_stream_mass_ww_kg'][scenario], "naturalgas","kg"))
        y["rail.mt_km"] = (cost["rail.mt_km"] * (SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 
                        SP.common_params['chlys_rail_mt_km'][scenario] +
                            cost["rail.mt_km"] * (
                                etoh_feed_stream_mass_kg/1000 * SP.common_params['etoh_distribution_rail'][scenario])) 
        y["flatbedtruck.mt_km"] = (cost["flatbedtruck.mt_km"] * (
                (SP.other_params[selectivity]['chlys_amount'][scenario]/1000) * 
                    SP.common_params['chlys_flatbedtruck_mt_km'][scenario]) +
                        cost["flatbedtruck.mt_km"] * (etoh_feed_stream_mass_kg/1000 * (
                            SP.common_params['etoh_distribution_truck'][scenario])))
        y["electricity.{}.kWh".format(SP.analysis_params['facility_electricity'])] = (
            cost["electricity.{}.kWh".format(SP.analysis_params['facility_electricity'])] * (
                SP.other_params[selectivity]['electricity_requirements'][scenario]))
        y["hcl.kg"] = cost["hcl.kg"] * SP.other_params[selectivity]['hcl.kg'][scenario]

        biorefinery_direct_ghg = hf.FuelCO2kg(hf.FuelConvertMJ(
                SP.other_params[selectivity]['ng_input_stream_mass_ww_kg'][scenario],"naturalgas","kg"), "naturalgas")

        results_kg_co2e = hf.TotalGHGEmissions(io_data, y, cost, 
                                               biorefinery_direct_ghg, SP.analysis_params['combustion_direct_ghg'],
                                              SP.analysis_params['time_horizon'])
           
        y_cred["electricity.{}.kWh".format(SP.analysis_params['facility_electricity'])] = -(
            cost["electricity.{}.kWh".format(SP.analysis_params['facility_electricity'])] * (
                SP.other_params[selectivity]['electricity_credit'][scenario]))
        
        
        results_kg_co2e_credit = hf.TotalGHGEmissions(io_data, y_cred, cost, 
                                                   biorefinery_direct_ghg, SP.analysis_params['combustion_direct_ghg'],
                                                  SP.analysis_params['time_horizon'])
        
        results_kg_co2e_credit_dict = results_kg_co2e_credit.set_index('products')['ghg_results_kg'].to_dict()
        
        results_kg_co2e_dict = results_kg_co2e.set_index('products')['ghg_results_kg'].to_dict()

        hf.AggregateResults(m, results_kg_co2e_dict, selectivity, scenario)

        
        m[scenario][selectivity] = m[scenario][selectivity] * 1000/27 # converting kg per kg results to g per MJ
        m[scenario][selectivity].loc['electricity_credit'] = (results_kg_co2e_credit[results_kg_co2e_credit['products'] == 
                                                'electricity.US.kWh']['ghg_results_kg'].iloc[0] * 1000/27)

In [21]:
m

{'avg':                            iHG-Projected  iHG-Current    waterwash
 electricity_credit            -15.280111    -1.797660    -0.000000
 Farming                         2.903830     2.903830     3.379564
 Transportation                183.106684   281.478985   289.019696
 Petroleum                     548.708845   969.610969   973.114943
 Electricity                   802.190888  1061.458250  1074.163519
 Chemicals_And_Fertilizers      99.194565   287.674899   299.049948
 Direct                          0.000000     0.000000    94.041719
 Other                           1.185185     1.703704     1.851852,
 'high':                            iHG-Projected  iHG-Current    waterwash
 electricity_credit             -1.797660    -1.797660    -0.000000
 Farming                         3.194213     3.194213     3.713196
 Transportation                200.786210   311.525344   312.210844
 Petroleum                     601.037932  1071.913120  1047.358998
 Electricity                   8

In [9]:
aggregated_data_avg = m['avg'][['waterwash', 'iHG-Current', 'iHG-Projected']].T
aggregated_data_low = m['low'][['waterwash', 'iHG-Current', 'iHG-Projected']].T
aggregated_data_high = m['high'][['waterwash', 'iHG-Current', 'iHG-Projected']].T

In [ ]:
aggregated_data_avg['electricity_credit'] = [0, -1.25, -15.22]
aggregated_data_low['electricity_credit'] = [0, -9.5, -25.54]
aggregated_data_high['electricity_credit'] = [0, -0.78, -2.1]

In [ ]:
aggregated_data_avg_pos = aggregated_data_avg.drop(['electricity_credit'],1)
aggregated_data_low_pos = aggregated_data_low.drop(['electricity_credit'],1)
aggregated_data_high_pos = aggregated_data_high.drop(['electricity_credit'],1)

In [ ]:
aggregated_data_avg_plot = aggregated_data_avg[list(reversed(aggregated_data_avg.columns.values))]

In [ ]:
aggregated_data_avg_plot

In [ ]:
error_min = (aggregated_data_low.sum(axis=1) - aggregated_data_avg_pos.sum(axis=1)).values*(-1)
error_max = (aggregated_data_high.sum(axis=1) - aggregated_data_avg_pos.sum(axis=1)).values
plt_errors = [error_min, error_max]

In [ ]:
import mpld3
mpld3.enable_notebook()
# mpld3.disable_notebook()

In [ ]:

plot = aggregated_data_avg_plot.plot(kind='bar', stacked = True, figsize=(9,6),
                                     colormap='PRGn_r',
                                     yerr={'Farming':plt_errors}, 
                                     error_kw= {'capsize':5, 'ecolor':'k', 'elinewidth':2, 'capthick':2, 'barsabove':True})
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.ylabel('gCO2 per MJ', fontsize = 18)
plt.ylim(-20,195)
plt.axhline(0, color='black', linewidth=2)
plot.set_axis_bgcolor("#EBEBEB")
plot.legend(loc='lower center', bbox_to_anchor=(1.25, 0.1),ncol=1, fancybox=True, shadow=True,fontsize=14)
with open('/Users/olga/Desktop/fig.html', 'w') as fn:
    fn.write(mpld3.fig_to_html(plt.gcf()))

In [ ]:
mpld3.fig_to_html(plt.gcf())